# Additional Thesis: Goda calculations


In [1]:
import math as ma
import numpy as np
import sympy

# Choice of method and design wave heights, periods etc

First, a choice between the standard design method and the new Eurocode 1 should be made. Based on the chosen Return Periods the corresponding wave heights and wave periods per method and limit state are used as input.

In [2]:
# Method to be used

method = "euro" #method can be "standard" or "eurocode"

In [3]:
# SLS

# Storm surge and wave stetup
if method=="standard":
    s_s_SLS = 0.74
else:
    s_s_SLS = 0.63
    
# Characteristic wave height (m) and Wave period (sec)
if method=="standard":
    H_13_SLS = 7.55 #RP = 70 years
    T_SLS = 11.00
else:
    H_13_SLS = 7.05 #RP = 40 years
    T_SLS = 10.71

In [4]:
# ULS

# Storm surge and wave stetup
if method=="standard":
    s_s = 0.99 
else:
    s_s = 0.70 #RP = 40 years
    
# Characteristic wave height (m) and Wave period (sec)
if method=="standard":
    H_13= 8.90 #RP = 475 years
    T = 11.70 
else:
    H_13 = 8.77 #RP = 400 years
    T = 11.66

# Dimensions of the cross section, water depths, slope

In [5]:
# Breakwater Characteristics and water depth

# Water Depth (m)
h_MSL = 35
# Water depth of the vertical wall (m)
d_MSL = 25

# Tidal range (m)
t_r = 0.15
# Sea Level Rise(m)
slr = 0.2

# Design Water Level(m)
h = h_MSL + t_r + s_s + slr
d = d_MSL + t_r + s_s + slr

# Submerged part of the Caisson (m) and Caisson width B (m)
h_sub = d
B = 26
chambers = 6
t_ext = 0.5
t_int = 0.2

# Freeboard (m)
h_c_start = 7
h_c = h_c_start - s_s - t_r - slr
h_c_SLS = h_c_start - s_s_SLS - t_r - slr

# Bottom slope (-)
tanθ = 1/80

# Incident wave angle (degrees)
β = 6

In [6]:
# Mass calculation (!assuming a given specific weight for the caisson!)
γ_con = 24  # KN/m3
γ_sand = 20  # KN/m3
γ_w = 10.25  # KN/m3

Wa =  γ_sand*(B-2*t_ext-(chambers-1)*t_int)*d + γ_con*(((2*t_ext+(chambers-1)*t_int)*d)+B*1+B*1.5+ (h_c - 1.5)*4)  #kg/m
W = (Wa - γ_w*(h_sub*B)) / 9.806 #N/m
#W

In [7]:
# Constants
π = ma.pi
ρ = 1.025 #t/m3 - Density of sea water
grav = 9.806 #m/s2 - Gravitational acceleration
μ = 0.6 # Friction coefficient

**Newton Raphson for the dispersion equation**

The following cell is related to the dispersion equation only, for which a Newton-Raphson method is used. It is not directly related to the Goda methodology.

In [8]:
# Code to solve the dispersion equation using Newton-Raphson method and store wave number k

h_nr = h
T_nr = T
x0 = 1
e = 0
N = 100
# Defining Function
def f(x):
    return 9.806*x*ma.tanh(h_nr*x)-(2*π/T_nr)**2
# Defining derivative of function
def g(x):
    return 9.806*(ma.tanh(h_nr*x)+h_nr*x*(1/(ma.cosh(h_nr*x)))**2)
# Implementing Newton Raphson Method
def newtonRaphson(x0,e,N):
    #print('\n\n*** NEWTON RAPHSON METHOD IMPLEMENTATION ***')
    step = 1
    flag = 1
    condition = True
    while condition:
        if g(x0) == 0.0:
            print('Divide by zero error!')
            break
        x1 = x0 - f(x0)/g(x0)
        #print('Iteration-%d, x1 = %0.6f and f(x1) = %0.6f' % (step, x1, f(x1)))
        x0 = x1
        step = step + 1
        if step > N:
            flag = 0
            break
        condition = abs(f(x1)) > e
    if flag==1:
        x0 = x1
      #  print('\nRequired root is: %0.5f' % x1)
    else:
        print('\nNot Convergent.') 
    return x1
# Input Section
#x0 = input('Enter Guess: ')
#e = input('Tolerable Error: ')
#N = input('Maximum Step: ')
# Converting x0 and e to float
x0 = float(x0)
e = float(e)
# Converting N to integer
N = int(N)
# Starting Newton Raphson Method
newtonRaphson(x0,e,N)


0.03483988039808753

In [9]:
k = _

In [10]:
# Easy equations

β = ma.radians(β)
H_max = 1.8*H_13 # Maximum wave height
h_star = 0.75*(1+np.cos(β))*H_max #m - Theoretical maximum level at which pressure is exerted
t = B/2 # Horizontal distance between the center of gravity of the caisson and its heel ##SOS: Assumption that t=B/2.
L = 2*π/k # Wave length calculation based on the wave number
L0 = grav*T**2/2/π # Deep water wave length
#Irribarren Number
ξ = tanθ / ((2*π*H_13/grav/T**2))**(0.5)

γ = 0.5 + 0.4 * ma.tanh(33*H_13/L0) # Wave breaking index according to Battje and Stive, 1985
#γ

In [11]:
# Wave breaking 
H_b = H_13
h_b = H_b / γ

In [12]:
# Goda method variables

a1 = 0.6 + 1/2*(2*k*h/np.sinh(2*k*h))**2
a2 = min((h_b - d)/(3*h_b)*(H_max/d)**2, 2*d/H_max)
a3 = 1 - (h_sub/h)*(1-(1/np.cosh(k*h)))

In [13]:
p1 = 1/2*(1+np.cos(β))*(a1+a2*(np.cos(β))**2)*ρ*H_max
p2 = p1 / (np.cosh(k*h))
p3 = a3*p1
if h_star>h_c:
    p4 = p1*(1-h_c/h_star)
else:
    p4 = 0

# Uplift
pu = 1/2*(1+np.cos(β))*a1*a3*ρ*H_max

In [14]:
# Stability
h_c_star = min(h_star, h_c)

F_H = 1/2*(p1+p3)*h_sub+1/2*(p1+p4)*h_c_star
F_U = 1/2*pu*B

# Moments
M_H = 1/6*(2*p1+p3)*(h_sub**2)+1/2*(p1+p4)*h_sub*h_c_star+1/6*(p1+2*p4)*(h_c_star**2)
M_U = 2/3*F_U*B
#F_U

In [15]:
# Safety Factors
SF_sliding = μ*(W-F_U)/(F_H)

SF_overturning = (W*t - M_U)/M_H

In [16]:
if SF_sliding<=1.2 or SF_overturning <= 1.2:
   print('Adjust the breakwaters design')
else:
    if SF_sliding > 1.2:
        print('The current design is \033[1msufficient against sliding\033[0m. The Safety Factor is:  %0.2f' % SF_sliding)
    else:
        print('The current design is \033[1mnot sufficient against sliding\033[0m.')
    if SF_overturning > 1.2:
        print('The current design is \033[1msufficient against overturning\033[0m. The Safety Factor is: %0.2f' % SF_overturning)
    else:
         print('The current design is \033[1mnot sufficient against overturning\033[0m.')
            
#SF_sliding 
#SF_overturning

The current design is sufficient against sliding. The Safety Factor is:  2.23
The current design is sufficient against overturning. The Safety Factor is: 2.76


In [17]:
# Net overturning moment
M_e = W*grav*t - M_H - M_U

# Nett vertical force on the subsoil
W_e = W*grav-F_U

# Eccentricity of the reaction force
t_e = M_e/W_e

In [18]:
if t_e<=(B/3):
    pe = 2*W_e/3/t_e
else:
    pe = 2*W_e/B*(2-3*(t_e/B))

# Foundation load Check
if pe>400:
    print('The foundation load is above the limit')
else:
    print('The bearing capacity of the soil is \33[1msufficient\33[1m.')
pe

The bearing capacity of the soil is sufficient.


370.6655427634354

**Berm stability**

For the berm stability the h' along with the corresponding L' and k' must be calculated.

In [19]:
#Dispersion equation for k'
h_berm = h_sub-2
T_nr2 = T
h_nr2 = h_berm
def f(x):
    return 9.806*x*ma.tanh(h_nr2*x)-(2*π/T_nr2)**2
# Defining derivative of function
def g(x):
    return 9.806*(ma.tanh(h_nr2*x)+h_nr2*x*(1/(ma.cosh(h_nr2*x)))**2)
newtonRaphson(x0,e,N)


0.03983411264649471

In [20]:
k_berm = _

In [21]:
# Berm Stability
Bm = 10
κ = (2*k_berm*h_berm/ma.sinh(2*k_berm*h_berm))*(ma.sin(k_berm*Bm))**2
Ns =  max(1.8, 1.3*(((1-κ)*h_berm)/(κ**(1/3)*H_13)+1.8*(ma.exp(-1.5*(1-κ)**2*h_berm/H_13/κ**(1/3)))))

ρ_c = 2500
ρ_w = 1025

Δ = (ρ_c - ρ_w) / ρ_w

# Required berm Dn50
Dn50_berm = H_13 / Δ / Ns
Dn50_berm = round(Dn50_berm,2)
Dn50_berm

0.83

**Overtopping calculations**

In [22]:
# Overtopping

# Acceptable overtopping discharge
q_acceptable = 1 #m3/m/s

# Using equation 9.8 of the Breakwaters Lecture Notes

over = h_c_SLS/H_13_SLS
if over<3.5 and over>0.1:
    a = 0.04
    b = 1.8
    q = a*ma.exp(-b*over)*ma.sqrt(grav*(H_13_SLS**3)) #m3/m/s
    print(round(q,3))
else:
    print('Equation not applicable for this combination of freeboard and Hs')
        
if q < q_acceptable:
    print('The design is sufficient for overtopping')
else:
    print('The overtopping discharge is higher than the acceptable one')

0.504
The design is sufficient for overtopping
